In [80]:
import pymongo
from pymongo import MongoClient
import json
from bs4 import BeautifulSoup
import re
import nltk # natural language toolkit
from urllib import request
from pprint import pprint
from datetime import datetime as dt
import zipfile
import requests
import io
import os
from pprint import pprint
from collections import defaultdict

import geopy
from geopy.geocoders import Nominatim

import time

### Download jsons and store

In [95]:
url_list = ['https://download.open.fda.gov/drug/event/2018q1/drug-event-0001-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0002-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0003-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0004-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0005-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0006-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0007-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0008-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0009-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0010-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0011-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0012-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0013-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0014-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0015-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0016-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0017-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0018-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0019-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0020-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0021-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0022-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0023-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0024-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0025-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0026-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0027-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0028-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0029-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0030-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0031-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0032-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q1/drug-event-0033-of-0033.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0001-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0002-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0003-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0004-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0005-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0006-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0007-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0008-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0009-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0010-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0011-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0012-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0013-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0014-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0015-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0016-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0017-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0018-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0019-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0020-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0021-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0022-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0023-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0024-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0025-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0026-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0027-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0028-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0029-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0030-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0031-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0032-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0033-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0034-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0035-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0036-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0037-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0038-of-0039.json.zip',
 'https://download.open.fda.gov/drug/event/2018q2/drug-event-0039-of-0039.json.zip']

In [49]:
for url in url_list:
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(r'C:\Data\FDA Data')

In [56]:
path = r'C:\Data\FDA Data'
filepaths = [path + '\\' + file for file in os.listdir(r'C:\Data\FDA Data')]

In [57]:
filepaths[0]

'C:\\Data\\FDA Data\\drug-event-0001-of-0033.json'

### Connect to MongoDB and import jsons

In [3]:
connection = MongoClient()
db = connection.kubrick

In [96]:
# Inserts negative events json data into MongoDB

#for fp in filepaths:
#    with open(fp) as f:
#        file_data = json.load(f)
#    
#    for res in file_data['results']:
#        db.fda.insert_one(res)

In [125]:
# Inserts recalls json data into MongoDB

#with open(r'C:\Data\FDA Enforcements\drug-enforcement-0001-of-0001.json') as f:
#    enforcement_data = json.load(f)
#
#for res in enforcement_data['results']:
#    db.fdarecalls.insert_one(res)

### Analysis
What are the key triggers and trends that lead to a product recall event occurring?

Estimate the impact of a recall, or group of recalls, to companies based on the trands identified.

In [50]:
#gets dict of firms and how many recalls they have made

q = [{'$project': {'recalling_firm': 1, '_id': 0}}, \
     {'$group': {'_id': '$recalling_firm', 'num_recalls': {'$sum': 1}}}, \
     #{'$match': {'num_recalls': {'$gt': 150}}}, \
     {'$sort': {'num_recalls': -1}}]

firms = defaultdict(dict)
for result in db.fdarecalls.aggregate(q):
    firms[result['_id']]['num_recalls'] = result['num_recalls']

In [15]:
firms

defaultdict(dict,
            {'Aidapak Services, LLC': {'num_recalls': 538,
              'top_words': [('mg', 1405),
               ('tablet', 1105),
               ('ndc', 944),
               ('pedigree', 923),
               ('exp', 910),
               ('ad', 602),
               ('potentially', 539),
               ('may', 538),
               ('mislabeled', 532),
               ('mixup', 531),
               ('labeling', 433),
               ('label', 433),
               ('capsule', 377),
               ('w', 314),
               ('hcl', 281)],
              'address': '14301 Se 1st St, , Vancouver, United States'},
             'Attix Pharmaceuticals': {'num_recalls': 470,
              'top_words': [('penicillin', 1398),
               ('repackaged', 940),
               ('products', 936),
               ('cross', 932),
               ('contamination', 932),
               ('lots', 470),
               ('recalled', 470),
               ('without', 470),
               ('dist

In [51]:
#makes dict of firms and all words contained in their reasons for recall
words_d = {}
for doc in db.fdarecalls.find({}, {'_id': 0, 'recalling_firm': 1, 'reason_for_recall': 1}):#.limit(5):
    words_d[doc['recalling_firm']] = words_d.get(doc['recalling_firm'], '') + ' ' + doc['reason_for_recall']

In [52]:
# makes list of top words in reasons for recall for each firm and puts them into firms dicitonary

words_count_d = {}
stop = nltk.corpus.stopwords.words('english')

for key in list(words_d.keys())[:]:
    #print(key)
    d = {}
    for w in words_d[key].split():
        #print (w.lower())
        w = re.sub('[^a-zA-Z\s\']+', '', w).lower()
        if w not in stop and w != '': # gets rid of stopwords
            d[w] = d.get(w, 0) + 1
            
    words_count_d[key] = sorted(d.items(), key = lambda x: x[1], reverse=True)[:15]
    
for key in list(firms.keys())[:]:
    firms[key]['top_words'] = words_count_d[key]

In [53]:
for doc in db.fdarecalls.find().limit(1):
    pprint(doc['recalling_firm'])

'Raritan Pharmaceuticals, Inc.'


In [56]:
firms['Raritan Pharmaceuticals, Inc.']

{'num_recalls': 15,
 'top_words': [('microbial', 24),
  ('contamination', 24),
  ('nonsterile', 12),
  ('products', 12),
  ('product', 12),
  ('recalled', 12),
  ('due', 12),
  ('possible', 12),
  ('c', 12),
  ('difficile', 12),
  ('discovered', 12),
  ('raw', 12),
  ('material', 12),
  ('cgmp', 3),
  ('deviations', 3)],
 'address': '8 Joanna Ct, East Brunswick, United States'}

In [37]:
[x*2 for x in (1,2,3) if x != 0]

[2, 4, 6]

In [55]:
for doc in db.fdarecalls.find():
    if firms[doc['recalling_firm']].get('address', 0) == 0:
        firms[doc['recalling_firm']]['address'] = ', '.join([doc['address_1'], doc['address_2'], doc['city'], doc['country']]).replace(' , ', ' ')
    #pprint(doc)
    #pprint(firms[doc['recalling_firm']]['address'])

In [82]:
for key in list(firms.keys())[:10]:
    try:
        firms[key]['latitude'] = location = geolocator.geocode(firms[key]['address']).latitude
        time.sleep(1)
        firms[key]['longitude'] = location = geolocator.geocode(firms[key]['address']).longitude
        time.sleep(1)
    except:
        continue

In [83]:
list(firms.values())[:5]

[{'num_recalls': 538,
  'top_words': [('mg', 1405),
   ('tablet', 1105),
   ('ndc', 944),
   ('pedigree', 923),
   ('exp', 910),
   ('ad', 602),
   ('potentially', 539),
   ('may', 538),
   ('mislabeled', 532),
   ('mixup', 531),
   ('labeling', 433),
   ('label', 433),
   ('capsule', 377),
   ('w', 314),
   ('hcl', 281)],
  'address': '14301 Se 1st St, Vancouver, United States',
  'latitude': -122.4686001},
 {'num_recalls': 470,
  'top_words': [('penicillin', 1398),
   ('repackaged', 940),
   ('products', 936),
   ('cross', 932),
   ('contamination', 932),
   ('lots', 470),
   ('recalled', 470),
   ('without', 470),
   ('distributed', 467),
   ('facility', 466),
   ('adequate', 466),
   ('separation', 466),
   ('could', 466),
   ('introduce', 466),
   ('potential', 466)],
  'address': '184 Front St E, unit 801, Toronto, Canada'},
 {'num_recalls': 383,
  'top_words': [('sterility', 764),
   ('lack', 382),
   ('assurance', 382),
   ('fda', 382),
   ('inspection', 382),
   ('identified',

In [84]:
geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.geocode("175 5th Avenue NYC")
print(location.address)
print((location.latitude, location.longitude))

GeocoderQuotaExceeded: HTTP Error 429: Too Many Requests

In [78]:
location = geolocator.geocode('6216 Highland Place Way Ste 101-A, Knoxville, United States')
#print(location.address)
print((location.latitude, location.longitude))

GeocoderQuotaExceeded: HTTP Error 429: Too Many Requests

In [ ]:
'address_1': '8 Joanna Ct',
 'address_2': '',
 'center_classification_date': '20121025',
 'city': 'East Brunswick',
 'classification': 'Class II',
 'code_info': 'Lot 15087, Exp 08/15',
 'country': 'United States',

In [75]:
latcount = 0
longcount = 0

for key in list(firms.keys()):
    if firms[key].get('latitude', 0) != 0:
        latcount +=1
    if firms[key].get('longitude', 0) != 0:
        longcount +=1

In [76]:
latcount

559

In [77]:
longcount

0